In [2]:
import sys
sys.path.append('app')

In [3]:
import os
import sqlite3
from database.crud import __tablefilter

# PATH = os.environ.get("DATA_PATH")
# PATH = "var/lib/data"

PATH = './data'

DATABASE_PATH = os.path.join(os.path.abspath(PATH), 'financedata.db')

connection = sqlite3.connect(DATABASE_PATH, check_same_thread=True)

cursor = connection.cursor()

In [7]:
from datetime import datetime 

def submissions_form_get(connection, filter:dict, columns:list=None, date_from:str=None, date_to:str=None, item_limit:int=1000) -> list[dict]:
    """
    Search for company filing by filter and return the result as a list of dictionary.
    
    :param:
        connection: sqlite connection object
        filter: The filter (in dict) to search for
        columns: List of columns to return (default is all columns)
        date_from: filter date from 
        date_to: filterr date to
    :return: Dictionary containing company info (return list if have many result) if found, otherwise None
    """
    cursor = connection.cursor()
    
    filter = {} if filter is None else list(filter.items())
    
    condition = []
    if filter is not None or len(filter) != 0:
        for q, v in filter:
            if v is not None:
                if hasattr(v, '__iter__') and not isinstance(v, str):
                    condition.append(f"{q} IN {str(tuple(v))}")
                else:
                    condition.append("{} = \"{}\"".format(q, v))
                
        condition = f" AND ".join(condition)    
    
        
    date_from = datetime.strftime(datetime.strptime(date_from, "%Y-%m-%d"), "%Y-%m-%d") if date_from is not None else None
    date_to = datetime.strftime(datetime.strptime(date_to, "%Y-%m-%d"), "%Y-%m-%d") if date_to is not None else None
    
    if date_from is not None:
        datefrom_cont = f"date(filing_date) > date(\"{date_from}\")"
        condition = condition + f" AND {datefrom_cont}" if len(condition) > 0 else datefrom_cont
    if date_to is not None:
        dateto_cont = f"date(filing_date) < date(\"{date_to}\")"
        condition = condition + f" AND {dateto_cont}" if len(condition) > 0 else dateto_cont
        
    if len(condition) == 0:
        condition = "TRUE"

    column = """
        ticker, companyInfo.cik, primary_doc_description, primary_docment,
        accession_number, filing_date, report_date, acceptance_date_time, 
        index_url, primary_docment_url, act, form
        """ if columns is None else ','.join(columns)
    
    query = f"""
        SELECT {column}
        FROM submissionForm INNER JOIN companyInfo ON submissionForm.cik = companyInfo.cik
        WHERE {condition}
        """
    
    print(query)
    
    q_res = cursor.execute(query)
    data = q_res.fetchmany(item_limit)
    
    columns = [ col[0] for col in q_res.description]
    
    if len(data) == 0:
        return None
    
    datadict = [ dict(zip(columns, r)) for r in data ]
    return datadict

submissions_form_get(
    connection, 
    filter={
        "ticker": None,
        "cik": None,
        "accession_number": None,
        "act": None,
        "form": ["10-Q", "10-K"] ,
    }, 
    columns=None, 
    date_from=None, 
    date_to=None, 
    item_limit=1000
    )


        SELECT 
        ticker, companyInfo.cik, primary_doc_description, primary_docment,
        accession_number, filing_date, report_date, acceptance_date_time, 
        index_url, primary_docment_url, act, form
        
        FROM submissionForm INNER JOIN companyInfo ON submissionForm.cik = companyInfo.cik
        WHERE form IN ('10-Q', '10-K')
        


[{'ticker': 'MHK',
  'cik': 851968,
  'primary_doc_description': '10-Q',
  'primary_docment': 'mhk-20240330.htm',
  'accession_number': '0000851968-24-000082',
  'filing_date': '2024-04-26',
  'report_date': '2024-03-30',
  'acceptance_date_time': '2024-04-26T12:08:53.000Z',
  'index_url': 'https://www.sec.gov/Archives/edgar/data/851968/000085196824000082/0000851968-24-000082-index.htm',
  'primary_docment_url': 'https://www.sec.gov/Archives/edgar/data/851968/000085196824000082/mhk-20240330.htm',
  'act': 34,
  'form': '10-Q'},
 {'ticker': 'MHK',
  'cik': 851968,
  'primary_doc_description': '10-K',
  'primary_docment': 'mhk-20231231.htm',
  'accession_number': '0000851968-24-000027',
  'filing_date': '2024-02-23',
  'report_date': '2023-12-31',
  'acceptance_date_time': '2024-02-23T17:16:41.000Z',
  'index_url': 'https://www.sec.gov/Archives/edgar/data/851968/000085196824000027/0000851968-24-000027-index.htm',
  'primary_docment_url': 'https://www.sec.gov/Archives/edgar/data/851968/00

In [31]:
from datetime import datetime

datetime.strftime(datetime.strptime("2024-01-1", "%Y-%m-%d"), "%Y-%m-%d")

'2024-01-01'

In [73]:
query = f"""
    SELECT DISTINCT form
    FROM submissionForm INNER JOIN companyInfo ON submissionForm.cik = companyInfo.cik
    """
    
q_res = cursor.execute(query)
q_res.fetchmany(500)

[(4,),
 ('11-K',),
 (144,),
 ('8-K',),
 ('SD',),
 ('10-Q',),
 ('ARS',),
 ('DEF 14A',),
 ('10-K',),
 ('SC 13G/A',),
 (3,),
 ('424B2',),
 ('FWP',),
 ('424B5',),
 ('S-3ASR',),
 ('PX14A6G',),
 (5,),
 ('SC 13G',),
 ('UPLOAD',),
 ('CORRESP',),
 ('3/A',),
 ('25-NSE',),
 ('S-8',),
 ('4/A',),
 ('CERT',),
 ('8-A12B',),
 ('SC 13D/A',),
 ('CERTNYS',),
 ('POSASR',),
 ('10-K/A',),
 ('DEFA14A',),
 ('NO ACT',),
 ('8-K/A',),
 ('424B7',),
 ('305B2',),
 ('424B3',),
 ('EFFECT',),
 ('S-3',),
 ('D',),
 ('424B4',),
 ('POS AM',),
 ('SEC STAFF LETTER',),
 ('S-1/A',),
 ('S-1',),
 ('DRSLTR',),
 ('DRS/A',),
 ('DRS',),
 ('PRE 14A',),
 ('SC TO-C',),
 ('SC 13D',),
 ('CT ORDER',),
 ('10-Q/A',),
 ('NT 10-K',),
 ('NT 10-Q',),
 ('SC TO-I/A',),
 ('SC TO-I',),
 (425,),
 ('S-4',),
 ('10-KT',),
 ('8-K12B/A',),
 ('8-K12B',),
 ('DEFM14A',),
 ('CERTNAS',),
 ('144/A',),
 ('REGDEX',),
 ('SC 14D9/A',),
 ('SC TO-T/A',),
 ('SC 14D9',),
 ('SC TO-T',),
 ('SC14D9C',),
 ('8-A12B/A',),
 ('PRRN14A',),
 ('PRER14A',),
 ('PREC14A',),
 ('DFA